In [1]:
%matplotlib inline

## 处理数据
PyTorch 有两个[处理数据的原语](https://pytorch.org/docs/stable/data.html)：“torch.utils.data.DataLoader”和“torch.utils.data.Dataset” `. ``Dataset`` 存储样本及其相应的标签，``DataLoader`` 围绕 ``Dataset`` 包装一个可迭代对象。



In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch 提供特定领域的库，例如 [TorchText](https://pytorch.org/text/stable/index.html)、[TorchVision](https://pytorch.org/vision/stable/index.html)、 和 [TorchAudio](https://pytorch.org/audio/stable/index.html)，所有这些都包含数据集。 在本教程中，我们将使用 TorchVision 数据集。

``torchvision.datasets`` 模块包含许多真实世界视觉数据的 ``Dataset`` 对象，例如 CIFAR、COCO（[完整列表在这里]（https://pytorch.org/docs/stable/torchvision/datasets. html))。 在本教程中，我们将使用 **FashionMNIST** 数据集。 每个 TorchVision ``Dataset`` 包括两个参数：``transform`` 和 ``target_transform`` 分别用于修改样本和标签。



In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将“Dataset”作为参数传递给“DataLoader”。 这在我们的数据集上包装了一个迭代，并支持自动批处理、采样、混洗和多进程数据加载。 这里我们定义了 64 的批量大小，即数据加载器迭代中的每个元素将返回一批 64 个特征和标签。



In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## 创建模型
为了在 PyTorch 中定义神经网络，我们创建了一个继承自 [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) 的类。 我们在 __init__ 函数中定义网络层，并在 forward 函数中指定数据如何通过网络。 为了加速神经网络中的操作，我们将其移至 GPU（如果可用）。



In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## 优化模型参数
为了训练模型，我们需要一个[损失函数](https://pytorch.org/docs/stable/nn.html#loss-functions>)和一个[优化器](https://pytorch.org/docs/ 稳定/optim.html）。



In [6]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在单个训练循环中，模型对训练数据集进行预测（分批提供给它），并反向传播预测误差以调整模型的参数。 


In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还可以根据测试数据集检查模型的性能，以确保它正在学习。

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程经过多次迭代（*epochs*）。 在每个时期，模型学习参数以做出更好的预测。 我们在每个时期打印模型的准确性和损失； 我们希望看到准确率随着每个 epoch 的增加而减少。



In [9]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298097  [    0/60000]
loss: 2.283960  [ 6400/60000]
loss: 2.265772  [12800/60000]
loss: 2.263425  [19200/60000]
loss: 2.240961  [25600/60000]
loss: 2.232002  [32000/60000]
loss: 2.246419  [38400/60000]
loss: 2.215907  [44800/60000]
loss: 2.220379  [51200/60000]
loss: 2.209777  [57600/60000]
Test Error: 
 Accuracy: 50.2%, Avg loss: 0.034368 

Epoch 2
-------------------------------
loss: 2.211811  [    0/60000]
loss: 2.176369  [ 6400/60000]
loss: 2.134772  [12800/60000]
loss: 2.157168  [19200/60000]
loss: 2.087879  [25600/60000]
loss: 2.087876  [32000/60000]
loss: 2.137016  [38400/60000]
loss: 2.067152  [44800/60000]
loss: 2.094769  [51200/60000]
loss: 2.081813  [57600/60000]
Test Error: 
 Accuracy: 50.6%, Avg loss: 0.031927 

Epoch 3
-------------------------------
loss: 2.085824  [    0/60000]
loss: 2.012798  [ 6400/60000]
loss: 1.937523  [12800/60000]
loss: 1.992881  [19200/60000]
loss: 1.872041  [25600/60000]
loss: 1.896021  [32000/600

最初的准确性不会很好（没关系！）。 尝试运行循环以获得更多的“epochs”或将“learning_rate”调整为更大的数字。 也可能是我们选择的模型配置可能不是此类问题的最佳配置（事实并非如此）。 后面的课程将深入研究适用于视力问题的模型形状。

保存模型
-------------
保存模型的常用方法是序列化内部状态字典（包含模型参数）。



In [10]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


加载模型
-----------------------------

加载模型的过程包括重新创建模型结构和加载
将状态字典放入其中。



In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

<All keys matched successfully>

该模型现在可用于进行预测。



In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
